In [24]:
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import math

import pymorphy2
import re
from collections import defaultdict

from tqdm import tqdm, tqdm_notebook
import bisect
from collections import namedtuple,defaultdict
import numpy as np


import lxml.html
import lxml.etree
import requests
import sqlite3


import time
import pickle
#MS SQL 
import pyodbc

In [3]:
conn = sqlite3.connect("news_base.db") 
cursor = conn.cursor()

In [25]:
conn = sqlite3.connect("news_base.db") 
cursor = conn.cursor()
#создание тестовой выборки, чтобы проверить качетсво кластеризации


d=[] #список документов для дальнейшей работы

doc_id = 0

stopWords = stopwords.words('russian')
stopWords.extend(['что', 'это', 'так', 'вот', 'быть','й', 'как', 'в', '—', 'к', 'на','свой', 'который', 'е', 'также','год'])

test_doc_to_clast = defaultdict(list)

categories = ['Культура','Спорт','Наука и техника'] 


#0 - культура , 1 -наука, 3 - спорт

for n_clast,  c in enumerate(categories):
    sql = "SELECT article FROM News where category = ? limit 400"
    cursor.execute(sql,[c] )
    
    for t in cursor.fetchall()[:50]:
        
        test_doc_to_clast[n_clast].append(doc_id)
        article = t[0]
        d.append(article)
        #print( n_clast, c, doc_id,article,"\n")
        doc_id+=1



d = sorted(d)
x = np.array(d) 
len(x)

150

In [4]:
a = 'Спорт'
b = 'Культура'
c='Наука и техника'
d=[]
test_doc_to_clast = defaultdict(list)
sql = "SELECT article, category FROM News where category = ? or category = ? or category = ? limit 300"
cursor.execute(sql, (a,b,c))
doc_id = 0
for t in cursor.fetchall():
        if t[1]==a:
            n_clast = 0
        elif t[1]==b:
            n_clast = 1
        else: n_clast = 2
            
        test_doc_to_clast[n_clast].append(doc_id)
        article = t[0]
        d.append(article)
        #print( n_clast, c, doc_id,article,"\n")
        doc_id+=1


In [5]:
#x = sorted(d)
x = np.array(d) 
len(test_doc_to_clast[2])

89

In [5]:
print(len(test_doc_to_clast[0]))
print(len(test_doc_to_clast[1]))
print(len(test_doc_to_clast[2]))

100
100
100


In [27]:
""" инвертированный индекс 
    используется для построения матриц вероятностей 
    """
stopWords = stopwords.words('russian')
stopWords.extend(['что', 'это', 'так', 'вот','ещё', 'свой' , 'быть','й', 'как', 'в', '—', 'к', 'на','свой', 'который', 'е','кстати', 'также'])

def parse_text(text):
    #print(type(text))
    words = (word for word in re.split('\W+', text) if (len(word) >0 )&( analyzer.normal_forms(word)[0].lower() not in stopWords ))
    lexems = (analyzer.normal_forms(word)[0] for word in words)
    
    return list(lexems)


class InvertedIndex:
    
    def __init__(self):
        self.dict = defaultdict(list)
        self.texts = dict()
        
    def add_document(self, text, doc_id):
        self.texts[doc_id] = text
        words = parse_text(text)
        
        
        word_to_entry = defaultdict(lambda: [])
        
        for pos, word in enumerate(words):
            
            doc_entry = word_to_entry[word]
            doc_entry.append(pos)
            
        for word, positions in word_to_entry.items():
            postings = self.dict[word]
            entry = DocEntry(doc_id, positions)
            
            i = bisect.bisect_left(postings, entry)
        
            postings.insert(i, entry)
            
    def get_postings(self, word):
        return self.dict[word]
    

        
"""Подсчет векторного произведения"""        
def mult_single(x,y):
    #conditions x>0 y>o
    return x*y

def mult_vect(x,y):
    mult = np.vectorize(mult_single)
    return np.sum(mult(x,y))

"""Dkl"""

def dkl_single(x,y):
    if x>0 and y>0:
     
        return x*math.log2(x/y)
    elif x==0: return 0.0
    else: return math.inf

def dkl(x,y):
    sum_dkl = np.vectorize(dkl_single)
    
    return np.sum(sum_dkl(x,y))

def div_matr(D, n, m,py_x,py_t) :
    for i in range(n):
        for j in range(m):
            Div[i,j] = dkl(py_x[:,i],py_t.transpose()[:,j]) 
            
    return Div
    
"""Djs"""
def djs(x,y):
    return 0.5 * dkl(x, (x+y)/2) + 0.5* dkl(y, (x+y)/2)

def djs_vect(Djs,m,pt_x,pt_x_new ):
    for i in tqdm(range(m)):
        Djs[i]= djs(pt_x[:,i], pt_x_new[:,i])
        
        #Djs[i]= djs(pt_x[i,:], pt_x_new[i,:])
    return Djs

"""P(t)"""
def f_pt(pt,m,px,pt_x):
    for i in range(m):
        pt[i] = mult_vect(px, pt_x[i,:])
    return pt

"""P(y|t)"""
def f_py_t(py_t, m,n,pt_x,pxy,pt):
    for i in range(m):
        for j in range(n):
            py_t[i,j] = mult_vect(pt_x[i,:],pxy[:,j])/pt[i]
            
    return py_t
"""Выводит список из  n элементов, соответсвующих  """
def main_words(n, r):
    l = []

    for i in range(n):
        l.append(r.argmax())
        r[r.argmax()] = 0
    return l


        
#show all numbers of docs from  cl,   
def show_text_docs(cl_num):
    show_main_words(cl_num)
    print("\n")
    docs = doc_clast[cl_num] # список документов кластера №cl_num
    for num in docs:
        print("№ документа: " + str(num) + " ТЕКСТ "+ str(x[num]))
    
    
    
def foo_z(pt_x):
    z = pt_x.sum(axis=0)
    return z


#w_d список документов со словом w
#d_cl список документов из кластера t 

#w_d,  d_cl
def f_N11( w_d,d_cl,n):
    k= 0
    for d in range(n):
        if (d in d_cl )& (d in w_d):
            k+=1
    return k
#not w_d, w_cl
def f_N01(w_d,w_cl,n):
    k= 0
    for d in range(n):
        if (d in w_cl )& (d not in w_d):
            k+=1
    return k
#not w_d, not w_cl
def f_N00( w_d,w_cl,n):
    k= 0
    for d in range(n):
        if (d not in w_cl )& (d not in w_d):
            k+=1
    return k

#w_d,   not w_cl
def f_N10( w_d,w_cl,n):
    k= 0
    for d in range(n):
        if (d not in w_cl )& (d in w_d):
            k+=1
    return k

def mutual_information(t):
    MI = []
    for word in tqdm(word_dict.items()):
        w  = word[1]
        
        w_d=[]
        for doc_id,pos in index.dict.get(w):
            w_d.append(doc_id)
        N = len(x)
        N11 = f_N11(w_d,doc_clast.get(t),len(x))
        N1 = len(w_d)
        N10 = f_N10(w_d,doc_clast.get(t),len(x))
        N0 = N - N1
        #print(N, N1, N0)
        N01 = f_N01(w_d,doc_clast.get(t),len(x))
        N00 = f_N00(w_d,doc_clast.get(t),len(x))
        if N10==0: N10=0.0001
        if N01==0: N01=0.0001
        if N11==0: N11=0.0001
        if N00==0: N00=0.0001
   
        #print(N11, N01, N10, N00)
        I = N11/N * math.log2(N*N11/(N1*N1)) + N01/N*math.log2(N * N01/(N0*N1)) +N10/N*math.log2(N * N10/(N0*N1))+N00/N*math.log2(N * N00/(N0*N0))
        MI.append(I)
    return MI
    
def main_words(n, r):
    l = []
    
    for i in range(n): 
        maxi = r.argmax()
        l.append((maxi,r[maxi]))
        r[maxi] = 0
    return l

In [117]:

def replaceZeroes(data):
    #print(data)
    data[data == 0] = 0.0001    
    return data

def replaceZeroeScalar(data):
    if data == 0:
        data = 0.0001   
    
    #print(data)
    return data

def IXT(px,pt_x,pt,m,n):
    IXT = 0
    for t in range(m):
        for x in range(n): #n len(x)
            #print(replaceZeroes([1,3,0,7]))
            a = px[x]*pt_x[t,:]*np.log2(replaceZeroes(pt_x[t,:]/replaceZeroeScalar(pt[t])))
            #print(a.sum())
            IXT+=a.sum()
    
    print("IXT = " + str(IXT))
    return IXT

def f_pty(pt_x,pxy,m,k):
    pty = np.zeros((m,k))#k - len words list
    for t in range(m):
        for y in range(k):
            
            pty[t,y] = mult_vect(pt_x[t,:], pxy[:,y])
            #print(pty[t,y])
    return pty
    
#через pty
def ITY_pty(pt_x,pxy,pt,m,k):
    ITY = 0
    pty = f_pty(pt_x,pxy,m,k)
    py = pxy.sum(axis=0)
    for t in range(m):
        for y in range(k):
            a = pty[t,:]*np.log2(replaceZeroes(pty[t,:]/replaceZeroeScalar(pt[t]*py[y])))
            ITY+=a.sum()   
            #print(ITY)
    
    print("ITY = " + str(ITY))
    return ITY

def ITY_py_t(py_t,pxy,pt, m, k):
    ITY = 0    
    py = pxy.sum(axis=0)
    for t in range(m):
        for y in range(k):
            #print(pt[t])
            a = pt[t] * py_t[t,:]*np.log2(replaceZeroes(py_t[t,:]/replaceZeroeScalar(py[y])))
            ITY+=a.sum()   
            #print(ITY)
    
    print("ITY = " + str(ITY))
    return ITY
    

In [70]:
#beta
beta = 2.3

#число кластеров
m = 3
#covergence parameter 
#для моих новостей 2.11е-03
e = 2.11e-04
e
len(x)

150

In [55]:
DocEntry = namedtuple('DocEntry',['doc_id','positions'])
"""INPUT"""

analyzer = pymorphy2.MorphAnalyzer()   
"""#Создание инвертированного индекса"""   
index = InvertedIndex()

for i, line in tqdm(enumerate(x)):
        doc_id = i
        text = line
        index.add_document(text,doc_id)

150it [00:24,  6.14it/s]


In [119]:
start_time = time.clock()

Func_L = []
        
#словарь слов: ключ - номер индекса , значение -словао
word_dict = {}
for i, word in enumerate(index.dict.keys()):
    word_dict[i] = word

"""#общее число слов в коллекции"""
summa = 0
for i in x:
    #print(len(word_tokenize(i)))
    summa+=len(word_tokenize(i))

"""P(x,y)"""    

pxy = np.zeros((len(x), len(index.dict.keys())))
for i, word in enumerate(index.dict.keys()):
    a = index.dict.get(word)
    for doc_id , postings in a:
        pxy[doc_id,i] = len(postings)/summa
"""print("pxy")
print(pxy)"""     
        
"""INITIALIZaTION""" 

"""P(t|x)"""

pt_x = np.random.rand(m, len(x))

z = pt_x.sum(axis=0)
pt_x = pt_x/z
#print(pt_x)

"""
pt_x = np.zeros((m, len(x)))
for row in np.arange(len(x)):
    i = np.random.randint(m)
    pt_x[i,row]=1 
 """       
"""задание вероятности P(x)"""    
px  = np.zeros((x.size))

for n, i in enumerate(x):
    b = len(word_tokenize(str(i)))
    #print(i, b, n)
    px[n] = b/summa
#print("px")
#print(px)
    


"""P(t)"""
pt = np.zeros(m)
for i in range(m):
    pt[i] = mult_vect(px,pt_x[i,:])
    
"""P(y|t)"""
py_t = np.zeros((m,len(index.dict.keys())) )

for i in range(m):
    for j in range(len(index.dict.keys())):
        py_t[i,j] = mult_vect(pt_x[i,:],pxy[:,j])/pt[i]
        

"""P(y|x)"""
py_x = np.zeros((len(index.dict.keys()),len(x)))


word_index=[]
for i, word in enumerate(index.dict.keys()):
    a = index.dict.get(word)
    word_index.append((i,word))
    for doc_id , postings in a:
        py_x[i,doc_id] = len(postings)/len(index.texts[doc_id].split())
        

      
        """LOOP"""
#print("y_x")
#print(py_x) 
Div = np.zeros((len(x), m)) #init div   
Div = div_matr(Div, len(x),m,py_x,py_t)
#new P(t|x)
#z = f_z(pt,py_x,py_t,beta)

pt_x_new =pt * np.exp(-beta * Div)
pt_x_new = pt_x_new.transpose()
#print("pt_x_new")
#print(pt_x_new)
z = foo_z(pt_x_new)
#print("z")
#print(z)
pt_x_new = pt_x_new/z
#print("pt_x_new")
#print(pt_x_new)

#init Djs
Djs = np.zeros(len(x))
Djs = djs_vect(Djs,len(x),pt_x,pt_x_new )

k = 0
"""print("pt_x")
print(pt_x)
print("pt")
print(pt)
print("py_t")
print(py_t)
"""
#print("DJS")
#print(Djs)
                                                                  
L = IXT(px,pt_x_new,pt,m, len(x)) - beta* ITY_py_t(py_t.transpose(),pxy,pt, m, len(index.dict.keys()))

#L = IXT(px,pt_x_new,pt,m, len(x)) -beta * ITY_pty(pt_x,pxy,pt,m,len(index.dict.keys()))
print(L)                                                                      
Func_L.append(L)
                                                                      
print (time.clock() - start_time, "seconds")
#while np.sum(Djs)/len(Djs)>=e: 
while Djs.max()>=e: 
    start_time = time.clock()
    print(k, "iteration")
    #переопрделяем 
    pt_x = pt_x_new 
    """print("pt_x")
    print(pt_x)"""
    #P(t)    
    #print("DJS")
    #print(Djs)
    pt = f_pt(pt,m,px,pt_x)
    """print("pt")
    print(pt)"""
    
    #P(y|t)
    py_t = f_py_t(py_t, m,len(index.dict.keys()),pt_x,pxy,pt) 
    """print(py_t)
    print(py_t)"""
                  
    #P(t|x) new
    pt_x_new = pt * np.exp(-beta * div_matr(Div, len(x),m,py_x,py_t)) 
    """print('pt_new')
    print(pt_x_new)"""
    pt_x_new = pt_x_new.transpose()
    z = foo_z(pt_x_new)
    pt_x_new = pt_x_new/z
    print("pt_x_new")
    print(pt_x_new[:,1])
    print(pt_x_new[:,100])
       
    Djs = djs_vect(Djs,len(x),pt_x,pt_x_new )
    print("DJS")
    #print(np.sum(Djs)/len(Djs))
    print(Djs.max())
    k+=1
    print("pt")
    print(pt)
    L = IXT(px,pt_x_new,pt,m, len(x)) - beta* ITY_py_t(py_t.transpose(),pxy,pt, m, len(index.dict.keys()))
    
    #L = IXT(px,pt_x_new,pt,m, len(x)) -beta * ITY_pty(pt_x,pxy,pt,m,len(index.dict.keys()))
    print("L = "+str(L))                                                                      
    Func_L.append(L)                                                                  
    print (time.clock() - start_time, "seconds")

# вектор с номерами кластеров, к которому документ предложит( вероятность больше всего)
vec = np.argmax(pt_x_new, axis=0)

#словарь: ключ - номер кластера, значение - номер документа
doc_clast = defaultdict(list)
for i,value in enumerate(vec):
        doc_clast[value].append(i)

        
#СЛОВАРЬ: ключ - номер кластера, значение - список и n самых важных слов

clasters_main_words = {}
n = 50
py_t_copy = py_t.copy()
for i, row in enumerate(py_t_copy):  
    
    clasters_main_words[i] = main_words(n, row) 
    #print(clasters_main_words[i])
    


100%|██████████████████████████████████████| 150/150 [00:00<00:00, 3844.01it/s]


IXT = 31.3623220578
ITY = 1.63051122167
27.612146248
6.217112976824865 seconds
0 iteration
pt_x_new
[ 0.80713255  0.1123458   0.08052165]
[ 0.27846229  0.12523952  0.59629819]


100%|██████████████████████████████████████| 150/150 [00:00<00:00, 1874.86it/s]


DJS
0.0209812028218
pt
[ 0.31383918  0.32320353  0.36295729]
IXT = 37.2240604594
ITY = 1.64384878823
L = 33.4432082464
4.58483616108424 seconds
1 iteration
pt_x_new
[ 0.83103111  0.09594554  0.07302335]
[ 0.26766623  0.15532695  0.57700682]


100%|██████████████████████████████████████| 150/150 [00:00<00:00, 5356.30it/s]


...

100%|██████████████████████████████████████| 150/150 [00:00<00:00, 4166.28it/s]


DJS
0.0363153723038
pt
[ 0.2993998   0.34083995  0.35976025]
IXT = 106.053226177
ITY = 1.93591734251
L = 101.60061629
4.104009969072649 seconds
5 iteration
pt_x_new
[ 0.94335212  0.02385062  0.03279727]
[ 0.18320816  0.17246808  0.64432375]


100%|██████████████████████████████████████| 150/150 [00:00<00:00, 6249.71it/s]


DJS
0.0351169358882
pt
[ 0.29628245  0.34419254  0.35952501]
IXT = 127.280251199
ITY = 2.04133835679
L = 122.585172978
4.691779738481273 seconds
6 iteration


KeyboardInterrupt: 

In [43]:
for i in doc_clast[0]:
    print(i,x[i]+'\n')

1 ['актер валерий николаев, отсидевший в сизо по делу о массовом дтп в центре москвы, вновь попался на нарушении правил дорожного движения. об этом во вторник, 5 апреля, сообщает lifenews.', 'по информации телеканала, в период с 1 по 3 апреля артиста 11 раз останавливали сотрудники дпс. так, 3 апреля он дважды превысил скорость более чем на 40 километров в час. за день до этого николаев совершил четыре нарушения, в числе которых выезд на полосу для маршрутного транспорта, игнорирование дорожных знаков и превышение скорости. 1 апреля актер попался на пяти нарушениях, в частности превышении и повторном превышении скорости (случаи были зафиксированы в московской области).', 'как отмечает lifenews, всего с момента выхода из-под ареста николаев проигнорировал пдд более 20 раз. между тем тверской суд москвы продолжает рассматривать нарушения, которые актер совершил ранее.', '8 марта николаев ', 'вышел', ' из московского сизо «бутырка», где он отбывал 10 суток административного ареста. актер 


16 ['астрофизики бенджамин бромли (из смитсоновской астрофизической обсерватории, сша) и скотт кеньон (из университета юты) оценили максимально допустимое число суперземель, которые могли бы возникнуть в солнечной системе. посвященный исследованию препринт авторы ', 'опубликовали', ' на сайте arxiv.org.', 'ученые полагают, что в гравитационном нестабильном кольце твердых тел, вращающихся вокруг солнца, могло возникнуть не более десяти суперземель. этот процесс может занять 100-250 миллионов лет в случае, если небесные тела формируются на расстоянии 250 астрономических единиц от солнца, и 1-2 миллиарда лет, если в три раза дальше. под суперземлями в своем исследовании ученые понимают планеты, которые до 50 раз тяжелее земли.', 'возникновение более десяти суперземель возможно за время, превышающее современный возраст солнечной системы (то есть маловероятно). в другой своей работе те же ученые ', 'рассмотрели', ' сценарий, в котором суперземли образуются недалеко от солнца, после чего не


24 ['британский актер бенедикт камбербэтч посетил нью-йоркский магазин jhu comic books в образе доктора стрэнджа. актер играет этого супергероя вселенной marvel в одноименном фильме и в перерыве между съемками решил заглянуть в книжный, не меняя одежды и не смывая грима.', 'картина о бывшем нейрохирурге, ставшем верховным магом земли, выйдет в прокат осенью 2016 года.']

30 ['в возрасте 96 лет в лондоне умер дуглас уилмер, исполнитель роли шерлока холмса в сериале «би-би-си» 1965-1968 годов. об этом сообщает the hollywood reporter.', 'он скончался в четверг, 31 марта, в больнице, расположенной в английском графстве саффолк, после непродолжительной болезни. диагноз не уточняется.', 'уилмер родился 8 января 1920 года в лондоне. учился на актера в королевской академии драматического искусства, какое-то время служил в северной африке. дебютировал на сцене в 1945 году, в 60-х стал играть в кино. согласно ', '«кинопоиску»', ', актер отметился примерно в 70 фильмах и сериалах.', 'в новом сер


73 ['медики из университета алабамы в бирмингеме (сша) определили полезную функцию сальника — складки внутренностной брюшины, богатой жировыми отложениями. соответствующее исследование опубликовано в журнале trends in immunology.', 'как показали ученые, сальник выполняет иммунные функции: жидкость, циркулирующая между органами брюшной полости, фильтруется тканями складки. по мнению авторов, сальник первым в организме реагирует на инфекцию (появляется фиброз из-за реакции на антигены), по его состоянию можно отследить начальный этап развития болезни.', 'между тем, по мнению ученых, сальник имеет и недостаток, способствуя распространению метастазов в организме (попадая в складку из брюшной полости, раковые клетки там и остаются). в частности, сальник выступает в качестве питательной среды для рака яичников и желудка.', 'специалисты предлагают использовать сальник в качестве индикатора начала воспалительного процесса в организме, а также в как потенциальную цель для профилактики метастаз


86 ['наса представило трехмерный снимок участка плутона, представляющего одну из самых странных форм рельефа, когда-либо наблюдаемых на поверхности небесного тела в солнечной системе. о клиновидных ландшафтах («змеиной коже») карликовой планеты ', 'сообщается', ' на сайте агентства.', 'на снимке показана область тартара дорса, которая расположена на восточной оконечности региона томбо (равнины в форме сердца). она простирается с севера на юг и представляет собой массив закругленных холмов высотой около 500 метров, отделенных друг от друга на 3-5 километров плоскими долинами.', 'ученые не знают происхождение этих геологических структур (клиновидных, змеинообразных или чешуйчатых ландшафтов). по одной из гипотез, они возникли в результате эрозии от сублимации водяного льда и отложений твердого метана, по другой они ', 'сформированы', ' метановыми ', 'клатратами', ' и представляют собой одни из самых древних структур в солнечной системе.', 'трехмерное изображение получено наложением двух


107 ['пятый канал опубликовал видео с места аварии, в которую попал режиссер эмир кустурица. продюсер кинематографиста мирко раденович сообщил национальной службе новостей (нсн), что кустурица, находившийся на пассажирском сиденье, не получил телесных повреждений. при этом пострадал водитель.', 'дтп ', 'произошло', ' в сербии на дороге между чачаком и белградом. водитель автомобиля mercedes не справился с управлением, и машина вылетела в кювет.']

109 ['режиссер лилли вачовски, ранее известная как энди, впервые появилась на публике после смены пола. она выступила на церемонии вручения премии glaad media awards, где получила награду за лучший драматический сериал («восьмое чувство»). вачовски произнесла речь о нелегком пути трансгендеров и необходимости бороться за право быть такой, какой хочется.', '«искусство никогда не бывает статичным, а идеи об идентичности и трансформации — важная часть нашей работы. но все идеи опираются на любовь. любовь — важнейшая вещь для транссексуалов. сли


139 ['ученые из университета джона хопкинса разработали новый способ борьбы с ожирением, который позволит избежать оперативного вмешательства. результаты работы исследователи ', 'представили', ' на ежегодной конференции общества интервенциональной радиологии.', 'новый метод называется «бариатрическая артериальная эмболизация». его мишенью является определенная часть желудка, фундус, который производит грелин — гормон, возбуждающий у человека аппетит. медики используют радиологические изображения и катетеры для доступа к кровеносным сосудам этой части желудка через прорезы в паху либо в запястье.', 'врач впрыскивает пациенту микроскопические бусинки, которые уменьшают приток крови к фундусу, тем самым подавляются гормональные сигналы голода, что приводит к снижению аппетита и потере веса.', 'методика была опробована на семи добровольцах с тяжелым ожирением с индексом массы тела от 40 до 60 килограммов на квадратный метр. для сравнения: у здорового человека индекс равен в среднем 21 кил

In [44]:
for i in Func_L:
    print(i)

-12929.6402888
-12967.2176142
-13086.4329558
-13291.7354052
-13597.4182395
-14017.3346322
-14558.4770705
-15199.0282843
-15864.9507736
-16465.2881804
-16946.4987537
-17310.5985911
-17589.441482
-17807.5187918
-17973.0472033
-18090.8617707
-18172.6841924
-18231.7697387
-18277.5808108
-18316.4196306
-18350.4350808
-18378.6124382
-18400.0386126
-18415.7534135
-18427.2057132
-18435.4504553
-18441.461046
-18446.1040474
-18449.9890194
-18453.4621416
-18456.6748424
-18459.6570802
-18462.3786694
-18464.7958208
-18466.879551
-18468.625672
-18470.0515841
-18471.1874219
-18472.0672468


In [104]:
# вектор с номерами кластеров, к которому документ предложит( вероятность больше всего)
vec = np.argmax(pt_x_new, axis=0)

#словарь: ключ - номер кластера, значение - номер документа
doc_clast = defaultdict(list)
for i,value in enumerate(vec):
        doc_clast[value].append(i)


In [20]:
pt_x[:,260]

array([ 0.03704451,  0.85432847,  0.10862703])

In [23]:
len(test_doc_to_clast[2]) #91 54 155

155

In [14]:
#подсчет статистики
l2  = test_doc_to_clast[0] 
l1 = doc_clast[2]

result=list(set(l1) & set(l2))
print(len(result))
#result

30


In [47]:
x[26]

"['американская модель и актриса эмили ратаковски снялась в клипе российского певца димы билана, сообщает lifenews.', 'клип на песню «неделимые» снимается в лос-анджелесе. соавтором композиции выступил михаил гуцериев, российский миллиардер и поэт-песенник. сумма гонорара эмили ратаковски не раскрывается.', 'ратаковски 24 года. она получила широкую известность после того, как снялась обнаженной в клипе американского певца робина тика blurred lines и love somebody группы maroon 5. она уже принимала участие в нескольких откровенных фотосессиях, снимок ратаковски появился на обложке турецкого gq. 31 марта телезвезда ким кардашьян ', 'поделилась', ' фотографией, на которой она и ратаковски топлес стоят у зеркала.', 'модель начала делать карьеру в кино, самый известный фильм с ее участием — триллер дэвида финчера «исчезнувшая». в ленте она сыграла любовницу главного героя энди фицджеральд.', 'в 2015 году журнал w magazine ', 'включил', ' ратаковски в десятку самых сексуальных современных мо

In [37]:
start_time = time.clock()
claster = 0
MI_0 = mutual_information(claster)
print (time.clock() - start_time, "seconds")

start_time = time.clock()
claster = 1
MI_1 = mutual_information(claster)
print (time.clock() - start_time, "seconds")


start_time = time.clock()
claster = 2
MI_2 = mutual_information(claster)
print (time.clock() - start_time, "seconds")

100%|█████████████████████████████████████| 8819/8819 [00:38<00:00, 228.61it/s]


40.44864591710302 seconds


100%|█████████████████████████████████████| 8819/8819 [00:32<00:00, 273.89it/s]


32.30015685336912 seconds


100%|█████████████████████████████████████| 8819/8819 [00:47<00:00, 185.48it/s]


47.656622134316876 seconds
